In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_scaled = StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h", 
                    "price_change_percentage_7d","price_change_percentage_14d", "price_change_percentage_30d",
                    "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"]])


In [ ]:
# Create a DataFrame with the scaled data
market_scaled_df = pd.DataFrame(market_scaled, columns=["price_change_percentage_24h", 
                    "price_change_percentage_7d","price_change_percentage_14d", "price_change_percentage_30d",
                    "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"])


# Copy the crypto names from the original data
market_scaled_df["coin_id"] = df_market_data.index


# Set the coinid column as index for market_scaled_df
market_scaled_df = market_scaled_df.set_index("coin_id")

# Display sample data
market_scaled_df.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k= list(range(1,11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:

# 1. Create a KMeans model using the loop counter for the n_clusters
for i in k:
    
# 2. Fit the model to the data using `df_market_data_scaled`
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(market_scaled_df)
    
# 3. Append the model.inertia_ to the inertia list
    inertia.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()


In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)


#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for k is 4, this means 4 is the optimal number of clusters.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(market_scaled_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_4 = model.predict(market_scaled_df)

# Print the resulting array of cluster values.
print(k_4)

In [ ]:
# Create a copy of the DataFrame
market_predictions_df = market_scaled_df.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
market_predictions_df["predicted_cluster"] = k_4

# Display sample data
market_predictions_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
market_predictions_df.hvplot.scatter(
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
# Color the graph points with the labels found using K-Means and 
    by='predicted_cluster',
# add the crypto name in the `hover_cols` parameter to identify 
    
# the cryptocurrency represented by each data point.
    hover_cols = 'coin_id')

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_data_pca = pca.fit_transform(market_predictions_df)
# View the first five rows of the DataFrame. 
market_data_pca[: 5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

In [ ]:
sum(pca.explained_variance_ratio_)


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:**  It is mentioned that 88% of the total variance is captured by three principal component analysis (PCA) variables. This high variance indicates a high level of predictability. Small variance, on the other hand, indicates that data points are clustered closely around the mean, making accurate predictions and pattern detection challenging for algorithms. In the context of cryptocurrency, high variance is desirable as it helps in predicting price trends in a volatile market. This volatility presents opportunities for traders to identify significant price movements and potentially achieve high returns on their investments.

In [ ]:
# Create a new DataFrame with the PCA data.
market_pca_df = pd.DataFrame(
    market_data_pca,
    columns = ["PCA1", "PCA2", "PCA3"]
)

# Copy the crypto names from the original data
market_pca_df['coin_id'] = df_market_data.index

# Set the coinid column as index
market_pca_df = market_pca_df.set_index("coin_id")

# Display sample data
market_pca_df.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
inertia2 = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
for i in k:
# 2. Fit the model to the data using `df_market_data_pca`
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(market_pca_df)
# 3. Append the model.inertia_ to the inertia list
    inertia2.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data2 = {"k": k, "inertia": inertia2}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow2 = pd.DataFrame(elbow_data2)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow2.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** The k value (number of clusters) remains the same as the original data. However, the modified data has a smaller inertia value of 43 compared to the original data with an inertia value of 79. A lower inertia value indicates that the data points are tightly clustered, which can be advantageous for identifying market regimes and behavioral patterns. This clear separation of clusters reduces noise in the data, making it easier to detect patterns and gain better insights. Moreover, tightly clustered data suggests similar characteristics and behaviors, potentially improving predictability.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the PCA data
model.fit(market_pca_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
clusters_predition = model.predict(market_pca_df)

# Print the resulting array of cluster values.
print(clusters_predition)

In [ ]:
# Create a copy of the DataFrame with the PCA data
clusters_predition_df= market_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
clusters_predition_df['clusters_predition'] = clusters_predition

# Display sample data
clusters_predition_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
clusters_predition_df.hvplot.scatter(
    
# `x="PC1"` and `y="PC2"`. 
    x="PCA1",
    y="PCA2",
# Color the graph points with the labels found using K-Means and 
    by = 'clusters_predition',
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
    hover_cols='coin_id'
)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
df_elbow.hvplot.line(x="k", y="inertia", title= "Elbow Curve 1", xticks=k) + df_elbow2.hvplot.line(x="k", y="inertia", title= "Elbow Curve 2",xticks=k)

In [ ]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!
market_predictions_df.hvplot.scatter( x='price_change_percentage_24h', y='price_change_percentage_7d',  title= "Cluster 1", by='predicted_cluster', hover_cols = 'coin_id') + clusters_predition_df.hvplot.scatter(x="PCA1", y="PCA2",title= "Cluster 2", by = 'clusters_predition', hover_cols='coin_id')


#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Elbow Curve Comparison:
Comparing the elbow curves of the PCA data (elbow curve 2) and the original data (elbow curve 1), it is observed that the PCA data has a lower inertia value. This suggests that by reducing the number of features, the sum of squared distances within the clusters has decreased. A lower inertia value indicates improved clustering performance and tighter grouping of data points within each cluster.

Cluster Scatter Plot Comparison:
Cluster Graph 1 (using original data): The clusters in this graph overlap, lacking clear separation or distinct clusters. This implies that the original data, with more features, may not have effectively captured the underlying patterns or structure of the data for clustering purposes.

Cluster Graph 2 (using PCA data): In contrast, the clusters in this graph generated using PCA data do not overlap. There is a clear separation between the clusters, and four distinct clusters can be identified. This indicates that reducing the number of features through PCA has potentially reduced noise in the data and revealed more distinct patterns. As a result, the clusters obtained are more meaningful and separable, providing better insight into the data.